# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [25]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [26]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [27]:
full_data_rows_list = [] 

for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
     
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [28]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [29]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [30]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS deproject 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [31]:
try:
    session.set_keyspace('deproject')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [32]:
#Table creation for event_data_hist: since the expected result is on session id and iteminsession, a table will be created with the session id and item session as Primark Key's
query = "CREATE TABLE IF NOT EXISTS event_data_hist"
query = query + "(artist text, song_title text, song_length float,session_id int,item_session int,\
                    PRIMARY KEY(session_id,item_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [33]:
#Table creation for event_data_user_hist: along with the Primary key's clustering with order by has been used inorder to order the data based on item session
query = "CREATE TABLE IF NOT EXISTS event_data_user_hist"
query = query + "(artist text, song_title text, first_name text, last_name text, session_id int, item_session int, user_id int,\
                    PRIMARY KEY((session_id,user_id),item_session))WITH CLUSTERING ORDER BY(item_session ASC);"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [34]:
#Table creation for extracting the users first name and last name: here clustering will be done on user id since it is mentioned as the second field in the Primary key:
query = "CREATE TABLE IF NOT EXISTS music_app_hist"
query = query + "(artist text, song_title text, first_name text, last_name text, user_id int, PRIMARY KEY(song_title,user_id));"
try:

    session.execute(query)
except Exception as e:
    print(e)



In [35]:
# File processing and data loading will be done in this block
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO event_data_hist (artist, song_title, song_length, session_id,item_session)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9],float(line[5]),int(line[8]),int(line[3])))
        
        query1 = "INSERT INTO event_data_user_hist (artist,song_title,first_name,last_name,session_id,item_session,user_id)"
        query1 = query1 + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query1, (line[0], line[9],line[1],line[4],int(line[8]),int(line[3]),int(line[10])))
        
        query2 = "INSERT INTO music_app_hist (artist,song_title,first_name,last_name,user_id)"
        query2 = query2 + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query2, (line[0], line[9],line[1],line[4],int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [37]:
#Query1 result
query = "select artist,song_title,song_length from event_data_hist WHERE session_id=338 and item_Session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [38]:
#Query2result
query = "select artist,song_title,first_name,last_name from event_data_user_hist WHERE session_id=182 and user_id=10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.first_name,row.last_name)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [39]:
#Query2result
query = "select first_name,last_name from music_app_hist WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name,row.last_name)
                               

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [40]:
## TO-DO: Drop the table before closing out the sessions
query_drop = 'Drop table event_data_hist'
query_drop1 = 'Drop table event_data_user_hist'
query_drop2 = 'Drop table music_app_hist'
try:
    session.execute(query_drop)
    session.execute(query_drop1)
    session.execute(query_drop2)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [41]:
session.shutdown()
cluster.shutdown()